## Business Understanding

- Goal: Build model to identify sets of items that are frequently bought together.

## Set Environment

In [1]:
import findspark
findspark.init()

In [2]:
import pandas as pd
import numpy as np

from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession

from pyspark.sql.functions import *
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.fpm import FPGrowth
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

In [3]:
spark = SparkSession.builder.appName('goods').getOrCreate()

## Loading Dataset

In [4]:
data = spark.read.csv('75000/75000i.csv', header = False, inferSchema = True)

In [5]:
data.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)



In [6]:
print('Total rows of dataset:', data.count())

Total rows of dataset: 266209


In [7]:
#rename the data
data = data.withColumnRenamed('_c0', 'receipt_id').withColumnRenamed('_c1', 'quantity').withColumnRenamed('_c2', 'product_id')

In [8]:
data.show(10, truncate = False)

+----------+--------+----------+
|receipt_id|quantity|product_id|
+----------+--------+----------+
|1         |1       |21        |
|1         |5       |11        |
|2         |1       |7         |
|2         |3       |11        |
|2         |4       |37        |
|2         |3       |45        |
|3         |5       |3         |
|3         |3       |42        |
|3         |3       |33        |
|4         |1       |5         |
+----------+--------+----------+
only showing top 10 rows



## Data Understanding

- There are 3 features in the data xxxxi.csv:
    - receipt_id: the transaction id
    - Quantity: the quantity of the product purchased per transaction
    - product_id: id of the product.

In [9]:
data.createOrReplaceTempView('order_product_data')

In [10]:
products = spark.sql('select distinct product_id from order_product_data')

In [11]:
products.count()

50

## Modeling and Evaluating

In this case, I will build in 2 cases, althought 2 case give the same result. But the second case will give the specific product rather than ID like in the forst case.

### Case 1: Model with product_id

In [12]:
rawData = spark.sql('select * from order_product_data')
baskets = rawData.groupBy('receipt_id').agg(collect_set('product_id')\
                                         .alias('items'))
baskets.createOrReplaceTempView('baskets')

- Group the data into receipt_id and all product_id with the following receipt_id will combined into a list called basket under the name items.
- This mean showed all the product in the basket per transaction

In [13]:
baskets.show(10, truncate = False)

+----------+----------------------+
|receipt_id|items                 |
+----------+----------------------+
|148       |[33, 27, 9, 46, 28, 4]|
|463       |[17, 14]              |
|471       |[9, 37, 34, 20]       |
|496       |[15, 6, 47, 26]       |
|833       |[12, 5, 21]           |
|1088      |[27, 35, 3, 18, 40]   |
|1238      |[19, 32, 18]          |
|1342      |[49, 17, 8]           |
|1580      |[12, 31, 36]          |
|1591      |[1, 19]               |
+----------+----------------------+
only showing top 10 rows



The minSupport and minConfidence will be set to the lowest due to the huge amount of transaction. This is one of the most popular properties of Association Rule problem.

In [14]:
fpgrowth = FPGrowth(itemsCol = 'items', minSupport = 0.003, minConfidence = 0.003)
model = fpgrowth.fit(baskets)

In [15]:
model.freqItemsets.show()

+--------+----+
|   items|freq|
+--------+----+
|     [7]|8193|
|    [45]|7700|
| [45, 7]|2367|
|    [28]|7556|
|[28, 45]| 387|
| [28, 7]| 383|
|    [18]|6987|
|[18, 28]| 393|
|[18, 45]| 318|
| [18, 7]| 321|
|     [4]|6948|
| [4, 28]| 465|
| [4, 45]| 372|
| [4, 18]| 402|
|  [4, 7]| 378|
|    [35]|6943|
|[35, 28]| 381|
| [35, 4]| 388|
|[35, 45]| 309|
|[35, 18]|3982|
+--------+----+
only showing top 20 rows



In [16]:
mostPopularItemInBasket = model.transform(baskets)

In [17]:
mostPopularItemInBasket.show(5, truncate =  False)

+----------+----------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|receipt_id|items                 |prediction                                                                                                                                                                            |
+----------+----------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|148       |[33, 27, 9, 46, 28, 4]|[35, 22, 45, 18, 42, 7, 14, 32, 1, 0, 16, 5, 3, 47, 37, 11, 15, 19, 44, 2, 17, 40, 12, 24, 49, 29, 41, 36, 23, 31, 43, 48, 39, 20, 21, 34, 30, 8, 38, 26, 6, 13, 10, 25]              |
|463       |[17, 14]              |[27, 28, 35, 4, 22, 45, 18, 42, 7, 32, 1, 0, 46, 16, 9, 5, 33, 3, 47, 37, 11, 15, 19, 44,

The result give alot of product recommended per transaction.

### Case2: Model with Product name (convert from product id)

To show the result with product_id, I will load the dataset contain the product information.

In [18]:
product = spark.read.csv('75000/goods.csv', header = True, inferSchema = True)

In [19]:
product.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Flavor: string (nullable = true)
 |-- Food: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Type: string (nullable = true)



In [20]:
product.show(3, truncate = False)

+---+-----------+------+-----+------+
|Id |Flavor     |Food  |Price|Type  |
+---+-----------+------+-----+------+
|0  |'Chocolate'|'Cake'|8.95 |'Food'|
|1  |'Lemon'    |'Cake'|8.95 |'Food'|
|2  |'Casino'   |'Cake'|15.95|'Food'|
+---+-----------+------+-----+------+
only showing top 3 rows



The flavor and food feature have the single quote in its value. So I need to replace them with "".

In [21]:
product = product.withColumn('Flavor', regexp_replace('Flavor', "\'", ""))
product = product.withColumn('Food', regexp_replace('Food', "\'", ""))

In [22]:
product.show(10)

+---+----------+------+-----+------+
| Id|    Flavor|  Food|Price|  Type|
+---+----------+------+-----+------+
|  0| Chocolate|  Cake| 8.95|'Food'|
|  1|     Lemon|  Cake| 8.95|'Food'|
|  2|    Casino|  Cake|15.95|'Food'|
|  3|     Opera|  Cake|15.95|'Food'|
|  4|Strawberry|  Cake|11.95|'Food'|
|  5|   Truffle|  Cake|15.95|'Food'|
|  6| Chocolate|Eclair| 3.25|'Food'|
|  7|    Coffee|Eclair|  3.5|'Food'|
|  8|   Vanilla|Eclair| 3.25|'Food'|
|  9|  Napoleon|  Cake|13.49|'Food'|
+---+----------+------+-----+------+
only showing top 10 rows



- I have to use Flavor and Food rather than choose 1 from 2 because the Flavor and Food have its own duplicate value if I ise 1 feature. For instance: Flavor have 2 Chocolate in its column, and Food have alot of cake category.
- The necessary option is to combine 2 feature: Chocolate Cake and Chocolate Eclair. So that it will not duplicate anymore.

In [23]:
product_data = product.select(concat_ws(' ',product.Flavor,product.Food).alias('product_name'),'Id', 'Flavor', 'Food', 'Price', 'Type')

In [24]:
product_data.show(5)

+---------------+---+----------+----+-----+------+
|   product_name| Id|    Flavor|Food|Price|  Type|
+---------------+---+----------+----+-----+------+
| Chocolate Cake|  0| Chocolate|Cake| 8.95|'Food'|
|     Lemon Cake|  1|     Lemon|Cake| 8.95|'Food'|
|    Casino Cake|  2|    Casino|Cake|15.95|'Food'|
|     Opera Cake|  3|     Opera|Cake|15.95|'Food'|
|Strawberry Cake|  4|Strawberry|Cake|11.95|'Food'|
+---------------+---+----------+----+-----+------+
only showing top 5 rows



In [25]:
product_data.createOrReplaceTempView('products')

In [26]:
rawData1 = spark.sql('''select p.product_name, o.receipt_id from products p inner join order_product_data o
                        where o.product_id = p.id''')
baskets1 = rawData1.groupBy('receipt_id').agg(collect_set('product_name')\
                                         .alias('items'))
#baskets1.createOrReplaceTempView('baskets1')

In [27]:
baskets1.head(3)

[Row(receipt_id=148, items=['Tuile Cookie', 'Strawberry Cake', 'Napoleon Cake', 'Chocolate Coffee', 'Cheese Croissant', 'Marzipan Cookie']),
 Row(receipt_id=463, items=['Berry Tart', 'Chocolate Tart']),
 Row(receipt_id=471, items=['Almond Twist', 'Chocolate Croissant', 'Pecan Tart', 'Napoleon Cake'])]

In [28]:
fpgrowth1 = FPGrowth(itemsCol = 'items', minSupport = 0.003, minConfidence = 0.003)
model1 = fpgrowth.fit(baskets1)

In [29]:
model1.freqItemsets.show()

+--------------------+----+
|               items|freq|
+--------------------+----+
|     [Coffee Eclair]|8193|
|        [Hot Coffee]|7700|
|[Hot Coffee, Coff...|2367|
|      [Tuile Cookie]|7556|
|[Tuile Cookie, Ho...| 387|
|[Tuile Cookie, Co...| 383|
|       [Cherry Tart]|6987|
|[Cherry Tart, Tui...| 393|
|[Cherry Tart, Hot...| 318|
|[Cherry Tart, Cof...| 321|
|   [Strawberry Cake]|6948|
|[Strawberry Cake,...| 465|
|[Strawberry Cake,...| 372|
|[Strawberry Cake,...| 402|
|[Strawberry Cake,...| 378|
|    [Apricot Danish]|6943|
|[Apricot Danish, ...| 381|
|[Apricot Danish, ...| 388|
|[Apricot Danish, ...| 309|
|[Apricot Danish, ...|3982|
+--------------------+----+
only showing top 20 rows



Same result but different items value: product_name insead of product id.

In [30]:
mostPopularItemInBasket1 = model1.transform(baskets1)

In [31]:
mostPopularItemInBasket1.show()

+----------+--------------------+--------------------+
|receipt_id|               items|          prediction|
+----------+--------------------+--------------------+
|       148|[Tuile Cookie, St...|[Hot Coffee, Cher...|
|       463|[Berry Tart, Choc...|[Lemon Tart, Marz...|
|       471|[Almond Twist, Ch...|[Marzipan Cookie,...|
|       496|[Vanilla Meringue...|[Lemon Tart, Marz...|
|       833|[Ganache Cookie, ...|[Lemon Tart, Chee...|
|      1088|[Cherry Tart, Ope...|[Cheese Croissant...|
|      1238|[Cherry Tart, Apr...|[Marzipan Cookie,...|
|      1342|[Single Espresso,...|[Lemon Tart, Marz...|
|      1580|[Apple Croissant,...|[Lemon Tart, Chee...|
|      1591|[Lemon Cake, Lemo...|[Marzipan Cookie,...|
|      1645|[Coffee Eclair, B...|[Hot Coffee, Tuil...|
|      1829|[Coffee Eclair, A...|[Lemon Tart, Rasp...|
|      1959|[Cherry Tart, Lem...|[Tuile Cookie, Ho...|
|      2122|[Truffle Cake, Go...|[Tuile Cookie, Ap...|
|      2142|[Bottled Water, B...|[Lemon Tart, Chee...|
|      236

In [32]:
mostPopularItemInBasket1.printSchema()

root
 |-- receipt_id: integer (nullable = true)
 |-- items: array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- prediction: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [33]:
mostPopularItemInBasket1.createOrReplaceTempView('popular_items')

In [34]:
DF_cast = mostPopularItemInBasket1.select('receipt_id', mostPopularItemInBasket1.items.cast(StringType()),
                                         mostPopularItemInBasket1.prediction.cast(StringType()))
DF_cast.printSchema()

root
 |-- receipt_id: integer (nullable = true)
 |-- items: string (nullable = false)
 |-- prediction: string (nullable = true)



Now the result already give alot of recommendation for every product.

In [35]:
DF_cast.head(3)

[Row(receipt_id=148, items='[Tuile Cookie, Strawberry Cake, Napoleon Cake, Chocolate Coffee, Cheese Croissant, Marzipan Cookie]', prediction='[Hot Coffee, Cherry Tart, Coffee Eclair, Apricot Danish, Orange Juice, Gongolais Cookie, Berry Tart, Apricot Croissant, Lemon Cake, Chocolate Cake, Blueberry Tart, Truffle Cake, Opera Cake, Vanilla Frappuccino, Almond Twist, Apple Pie, Blackberry Tart, Lemon Tart, Bottled Water, Casino Cake, Chocolate Tart, Lemon Lemonade, Apple Tart, Lemon Cookie, Single Espresso, Walnut Cookie, Raspberry Lemonade, Apple Danish, Raspberry Cookie, Apple Croissant, Green Tea, Cherry Soda, Blueberry Danish, Pecan Tart, Ganache Cookie, Chocolate Croissant, Almond Croissant, Vanilla Eclair, Almond Bear Claw, Vanilla Meringue, Chocolate Eclair, Apricot Tart, Almond Tart, Chocolate Meringue]'),
 Row(receipt_id=463, items='[Berry Tart, Chocolate Tart]', prediction='[Lemon Tart, Marzipan Cookie, Cheese Croissant, Lemon Cake, Tuile Cookie, Almond Twist, Apricot Danish, Bl

DF_cast = DF_cast.write.csv('recommended_goods.csv')